# AbdolazizsalimiFinal980292626

## About DataSet:

### Abstract: 


Contrast-enhanced spectral mammography (CESM) is a relatively recent imaging modality with increased diagnostic accuracy compared to digital mammography (DM). New deep learning (DL) models were developed that have accuracies equal to that of an average radiologist. However, most studies trained the DL models on DM images as no datasets exist for CESM images. We aim to resolve this limitation by releasing a Categorized Digital Database for Low energy and Subtracted Contrast Enhanced Spectral Mammography images (CDD-CESM) to evaluate decision support systems. The dataset includes 2006 images, with an average resolution of 2355 × 1315, consisting of 310 mass images, 48 architectural distortion images, 222 asymmetry images, 238 calcifications images, 334 mass enhancement images, 184 non-mass enhancement images, 159 postoperative images, 8 post neoadjuvant chemotherapy images, and 751 normal images, with 248 images having more than one finding. This is the first dataset to incorporate data selection, segmentation annotation, medical reports, and pathological diagnosis for all cases. Moreover, we propose and evaluate a DL-based technique to automatically segment abnormal findings in images.

![a:Low-energy | b:High-energy | c:subtracted-Image](Lowenergy-Highenergy-Sub.png)


* (a) :Low-energy 
* (b) :High-energy
* (c) :subtracted-Image

### load Radiology manual file :

In [3]:
import pandas as pd

In [4]:
path = r'../Radiology-manual-annotations.xlsx'
manual = pd.read_excel(path)

In [5]:
df = manual.copy()

In [6]:
df.head()

,Image_name,Patient_ID,Side,Type,Age,Breast density (ACR),BIRADS,Findings,View,Tags,Machine,Pathology Classification/ Follow up
0,P1_L_DM_MLO,1,L,DM,46,C,4,Architectural distortion,MLO,"suspicious, distortion",1,Malignant
1,P1_L_CM_MLO,1,L,CESM,46,_,4,Heterogenous non mass enhancement,MLO,"suspicious, non mass",1,Malignant
2,P2_R_DM_CC,2,R,DM,31,C,3$2,Multiple equal density masses$Benign macrocal...,CC,"benign, masses, macrocalcification",2,Benign
3,P2_R_DM_MLO,2,R,DM,31,C,3$2,Multiple equal density masses$Benign macrocalc...,MLO,"benign, masses, macrocalcification",2,Benign
4,P2_R_CM_CC,2,R,CESM,31,_,3,Multiple homogenously enhancing masses,CC,"benign, masses, homogenous",2,Benign


In [7]:
df.isnull().sum()

Image_name                             0
Patient_ID                             0
Side                                   0
Type                                   0
Age                                    0
Breast density (ACR)                   0
BIRADS                                 0
Findings                               0
View                                   0
Tags                                   0
Machine                                0
Pathology Classification/ Follow up    0
dtype: int64

In [8]:
# Rename the column
df.rename(columns={'Pathology Classification/ Follow up': 'PathologyClassification'}, inplace=True)

In [9]:
df.columns

Index(['Image_name', 'Patient_ID', 'Side', 'Type', 'Age',
       'Breast density (ACR)', 'BIRADS', 'Findings', 'View', 'Tags', 'Machine',
       'PathologyClassification'],
      dtype='object')